# Etapa 1: Importação das bibliotecas

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

In [ ]:
!pip install bert-for-tf2

In [ ]:
!pip install sentencepiece

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
import tensorflow_hub as hub

In [ ]:
from tensorflow.keras import layers
import bert

# Etapa 2: Carregamento da base de dados

In [ ]:
drive.mount('/content/drive')

In [ ]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ProjetoIA/training.1600000.processed.noemoticon.csv',
                   header = None,
                   names = cols,
                   engine = 'python',
                   encoding='latin-1')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.drop(['id', 'date', 'query', 'user'],
          axis=1,
          inplace=True)